In [ ]:
%load_ext autoreload
%autoreload 2

from geomstats.geometry.reparametrizable_curves import *
from geomstats.geometry.euclidean import Euclidean
from sklearn.manifold import Isomap
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from scipy.stats import special_ortho_group
from numpy.testing import assert_almost_equal

In [ ]:
t_ = np.linspace(0,50,501)
# t_ = list(range(52))

In [ ]:
repcurve1 = ReparametrizableCurve({t:np.array((np.cos(t), 
                                               np.sin(t), 
                                               t)
                                             ) for t in t_}, 't', Euclidean(dim=3))

repcurve1_pts = np.asarray(list(repcurve1.t_curve.values()))
repcurve2_pts_df = pd.DataFrame(repcurve1_pts, columns=['x', 'y', 'z'])
fig = px.scatter_3d(repcurve2_pts_df, x='x', y='y', z='z', 
                    color=repcurve2_pts_df.index, color_continuous_scale='jet')
fig.update_layout(scene = dict(aspectmode='data',
                               aspectratio=dict(x=1, y=1, z=1),# auto, cube, data, manual
                              ),)
fig.show(renderer='notebook')

In [ ]:
repcurve1_df = pd.DataFrame(repcurve1.curvatures(2,).transpose(), 
                        columns=[f'c{jj+1}' for jj in range(2)])
pd.options.plotting.backend = "plotly"
repcurve1_df.plot(x=repcurve1_df.index, y=[f'c{jj+1}' for jj in range(2)])

In [ ]:
RR = special_ortho_group.rvs(repcurve1_pts.shape[1])
assert_almost_equal(np.eye(repcurve1_pts.shape[1]), np.matmul(RR.transpose(), RR))
assert_almost_equal(np.linalg.det(RR), 1)

rot_repcurve1_pts = np.matmul(RR, repcurve1_pts.transpose()).transpose() \
                    + np.array([np.random.uniform(-100, 100, repcurve1_pts.shape[1])])

rot_repcurve1_pts_df = pd.DataFrame(rot_repcurve1_pts, columns=['x', 'y', 'z'])
fig = px.scatter_3d(rot_repcurve1_pts_df, x='x', y='y', z='z', 
                    color=rot_repcurve1_pts_df.index, color_continuous_scale='jet')
fig.show(renderer='notebook')

rot_repcurve1_rc = ReparametrizableCurve(
    {t:rot_repcurve1_pts[idx,:] for idx, t in enumerate(t_)}, 't', Euclidean(dim=3))

rot_repcurve1_df = pd.DataFrame(rot_repcurve1_rc.curvatures(2).transpose(), 
                        columns=[f'c{jj+1}' for jj in range(2)])
# fig = px.plot(x=rot_repcurve1_df.index, y=[f'c{jj+1}' for jj in range(2)])
# fig.update_layout(scene = dict(aspectmode='data',
#                                aspectratio=dict(x=1, y=1, z=1),# auto, cube, data, manual
#                               ),)
# fig.show(renderer="notebook")
pd.options.plotting.backend = "plotly"
rot_repcurve1_df.plot(x=rot_repcurve1_df.index, y=[f'c{jj+1}' for jj in range(2)])

In [ ]:
noisy_repcurve1_pts = repcurve1_pts + np.random.normal(0, .1, repcurve1_pts.shape)
# noisy_repcurve1_pts = np.array([np.convolve(noisy_repcurve1_pts[:,jj], 
#                                             np.ones(10)/10., mode='same').transpose()
#                                 for jj in range(noisy_repcurve1_pts.shape[1])]).transpose()

noisy_repcurve1_pts_df = pd.DataFrame(noisy_repcurve1_pts, columns=['x', 'y', 'z'])
fig = px.scatter_3d(noisy_repcurve1_pts_df, x='x', y='y', z='z', 
                    color=noisy_repcurve1_pts_df.index, color_continuous_scale='jet')
fig.show(renderer='notebook')

noisy_repcurve1_rc = ReparametrizableCurve(
    {t:noisy_repcurve1_pts[idx,:] for idx, t in enumerate(t_)}, 't', Euclidean(dim=3))

noisy_repcurve1_df = pd.DataFrame(
    noisy_repcurve1_rc.curvatures(2, smooth_curvatures={
        'method':'boxcar',
        'win_width':20,
    }).transpose(), 
    columns=[f'c{jj+1}' for jj in range(2)])#.rolling(20).mean()
# fig = px.plot(x=rot_repcurve1_df.index, y=[f'c{jj+1}' for jj in range(2)])
# fig.update_layout(scene = dict(aspectmode='data',
#                                aspectratio=dict(x=1, y=1, z=1),# auto, cube, data, manual
#                               ),)
# fig.show(renderer="notebook")
pd.options.plotting.backend = "plotly"
noisy_repcurve1_df.plot(x=noisy_repcurve1_df.index, y=[f'c{jj+1}' for jj in range(2)])

In [ ]:
repcurve2 = ReparametrizableCurve({t:np.array((np.cos(t), 
                                               np.sin(t), 
                                               np.sin(2*t), 
                                               np.cos(2*t), 
                                               np.sqrt(t*np.log(t+1.1)))
                                             ) for t in t_}, 't', Euclidean(dim=5))

repcurve2_pts = np.asarray(list(repcurve2.t_curve.values()))
isomap = Isomap(n_components=3)
repcurve2_pts_lowdim = isomap.fit_transform(repcurve2_pts)
repcurve2_pts_lowdim_df = pd.DataFrame(repcurve2_pts_lowdim, columns=['x', 'y', 'z'])
fig = px.scatter_3d(repcurve2_pts_lowdim_df, x='x', y='y', z='z', 
                    color=repcurve2_pts_lowdim_df.index, color_continuous_scale='jet')
# fig.update_layout(scene = dict(aspectmode='data',
#                                aspectratio=dict(x=1, y=1, z=1),# auto, cube, data, manual
#                               ),)
fig.show(renderer='notebook')

In [ ]:
crvts_df = pd.DataFrame(repcurve2.curvatures(4).transpose(), 
                        columns=[f'c{jj+1}' for jj in range(4)])
crvts_lowdim_df = pd.DataFrame(isomap.fit_transform(crvts_df.iloc[1:-2, :].values),
                               columns=[f'c{jj+1}' for jj in range(3)]
                              )
# fig = px.scatter_3d(crvts_lowdim_df, x='c1', y='c2', z='c3', 
#                     color=crvts_lowdim_df.index, color_continuous_scale='jet')
pd.options.plotting.backend = "plotly"
crvts_df.plot(x=crvts_df.index, y=[f'c{jj+1}' for jj in range(4)])

In [ ]:
get_torus_know = lambda p, q: {t:np.array(
    ((np.cos(q*t)+4)*np.cos(p*t), 
     (np.cos(q*t)+4)*np.sin(p*t), 
                    -np.sin(q*t))) for t in np.linspace(0,2*np.pi,200)}

torusknot_rc = ReparametrizableCurve(get_torus_know(2,3), 't', Euclidean(dim=3))

torusknot_vals = np.asarray(list(torusknot_rc.t_curve.values()))

fig = px.scatter_3d(x=torusknot_vals[:,0], 
                    y=torusknot_vals[:,1], 
                    z=torusknot_vals[:,2], 
                    color=range(torusknot_vals.shape[0]), 
                    color_continuous_scale='jet')
fig.update_layout(scene = dict(aspectmode='data',
                               aspectratio=dict(x=1, y=1, z=1),# auto, cube, data, manual
                              ),)
fig.show(renderer='notebook')

In [ ]:
crvts_df = pd.DataFrame(torusknot_rc.curvatures(2).transpose(), 
                        columns=[f'c{jj+1}' for jj in range(2)])
pd.options.plotting.backend = "plotly"
crvts_df.plot(x=crvts_df.index, y=[f'c{jj+1}' for jj in range(2)])

In [ ]:
RR = special_ortho_group.rvs(torusknot_vals.shape[1])
assert_almost_equal(np.eye(torusknot_vals.shape[1]), np.matmul(RR.transpose(), RR))
assert_almost_equal(np.linalg.det(RR), 1)

rot_torusknot_vals = np.matmul(RR, torusknot_vals.transpose()).transpose() \
                    + np.array([np.random.uniform(-100, 100, torusknot_vals.shape[1])])

fig = px.scatter_3d(x=rot_torusknot_vals[:,0], 
                    y=rot_torusknot_vals[:,1], 
                    z=rot_torusknot_vals[:,2], 
                    color=range(rot_torusknot_vals.shape[0]), 
                    color_continuous_scale='jet')
fig.update_layout(scene = dict(aspectmode='data',
                               aspectratio=dict(x=1, y=1, z=1),# auto, cube, data, manual
                              ),)
fig.show(renderer='notebook')

rot_torusknot_rc = ReparametrizableCurve(
    {t:rot_torusknot_vals[idx,:] for idx, t in enumerate(np.linspace(0,2*np.pi,200))}, 't', Euclidean(dim=3))
crvts_df = pd.DataFrame(rot_torusknot_rc.curvatures(2).transpose(), 
                        columns=[f'c{jj+1}' for jj in range(2)])
pd.options.plotting.backend = "plotly"
crvts_df.plot(x=crvts_df.index, y=[f'c{jj+1}' for jj in range(2)])

In [ ]:
noisy_torusknot_vals = torusknot_vals + np.random.normal(0, .1, torusknot_vals.shape)

fig = px.scatter_3d(x=noisy_torusknot_vals[:,0], 
                    y=noisy_torusknot_vals[:,1], 
                    z=noisy_torusknot_vals[:,2], 
                    color=range(noisy_torusknot_vals.shape[0]), 
                    color_continuous_scale='jet')
fig.update_layout(scene = dict(aspectmode='data',
                               aspectratio=dict(x=1, y=1, z=1),# auto, cube, data, manual
                              ),)
fig.show(renderer='notebook')

noisy_torusknot_rc = ReparametrizableCurve(
    {t:noisy_torusknot_vals[idx,:] for idx, t in enumerate(np.linspace(0,2*np.pi,200))}, 't', Euclidean(dim=3))
crvts_df = pd.DataFrame(
    noisy_torusknot_rc.curvatures(2, smooth_curvatures={
        'method':'boxcar',
        'win_width':15,
    }).transpose(), 
    columns=[f'c{jj+1}' for jj in range(2)])#.rolling(20).mean()
pd.options.plotting.backend = "plotly"
crvts_df.plot(x=crvts_df.index, y=[f'c{jj+1}' for jj in range(2)])